# Обучение итоговой модели и предсказание для public test


В данной тетрадке, я загружаю данные по всем пользователям полученные на предыдущем этапе (см. mts_ml_cup_sibrikova_main_body.ipynb): Для итогового пhедсказания мною была выбрана модель catboost Classifier и для предсказания пола и для предсказания возраста, так как они покаpали наилучшее качество на валидационно выборке. А так же среди ранее проведенной кластеризации сайтов (см.mts_ml_cup_sibrikova_url_claster.ipynb) в варианте 50 кластеров, так как по мере увеличения кластеров кач-во предсказания таргетов менялось не значительно.

*Также ниже преставлен код подбора гиперпараметров итоговых моделей*

In [148]:
#Импортируем библиотеки
import sys
import os
import warnings
os.environ['OPENBLAS_NUM_THREADS'] = '1'

import pandas as pd
import numpy as np
import time
import pyarrow.parquet as pq
import pyarrow as pa
import scipy
import implicit
import bisect
import sklearn.metrics as m
from catboost import CatBoostClassifier, CatBoostRegressor, Pool
from sklearn.model_selection import train_test_split
from sklearn.calibration import calibration_curve, CalibratedClassifierCV
from xgboost import XGBClassifier

import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px

from sklearn.preprocessing import StandardScaler, OneHotEncoder,OrdinalEncoder

from sklearn.linear_model import LogisticRegression,LinearRegression
from sklearn.ensemble import RandomForestClassifier,RandomForestRegressor
import lightgbm as lgb

warnings.simplefilter(action='ignore', category=pd.errors.PerformanceWarning)

from sklearn.metrics import f1_score

In [152]:
# путь к папке с файлу данных
LOCAL_DATA_PATH = './context_data'

# параметр
SPLIT_SEED = 42


# таргеты
TARGET_FILE = 'public_train.pqt'

# ID клиентов по которым делаем сабмит
SUBMISSION_FILE = 'submit_2.pqt'

#файл с параметрами пользователей
info_user = 'info_user.parquet'

#файл с результатами кластеризации сайтов 
df2_high_50 = 'df2_high_50.parquet'

In [153]:
%%time

# Таргеты
targets = pq.read_table(f'{LOCAL_DATA_PATH}/{TARGET_FILE}')
usr_targets = targets.to_pandas()

info_user = pq.read_table(f'{info_user}').to_pandas()
df2_high_50 = pq.read_table(f'{df2_high_50}')
id_to_submit = pq.read_table(f'{LOCAL_DATA_PATH}/{SUBMISSION_FILE}').to_pandas()

CPU times: user 1.12 s, sys: 2.73 s, total: 3.85 s
Wall time: 1.77 s


In [154]:
%%time
url_set = set(df2_high_50.select(['url']).to_pandas()['url'])
print(f'{len(url_set)} urls')
url_dict = {url: idurl for url, idurl in zip(url_set, range(len(url_set)))}
usr_set = set(df2_high_50.select(['user_id']).to_pandas()['user_id'])
print(f'{len(usr_set)} users')
usr_dict = {usr: user_id for usr, user_id in zip(usr_set, range(len(usr_set)))}

values = np.array(df2_high_50.select(['request_cnt_sum']).to_pandas()['request_cnt_sum'])
rows = np.array(df2_high_50.select(['user_id']).to_pandas()['user_id'].map(usr_dict))
cols = np.array(df2_high_50.select(['url']).to_pandas()['url'].map(url_dict))
mat = scipy.sparse.coo_matrix((values, (rows, cols)), shape=(rows.max() + 1, cols.max() + 1))
mat = pd.DataFrame.sparse.from_spmatrix(mat)
#Составлем разреженную матрицу и переводим в dataframe


dic={}
for i in mat.columns:
    dic[i]=str(i)+'high'

mat=mat.rename(columns=dic)

mat.index=\
np.array(df2_high_50.select(['user_id']).to_pandas()['user_id'].unique())


info_user50=info_user.join(mat)
info_user50=info_user50.fillna(0)

987 urls
415274 users
CPU times: user 28.7 s, sys: 0 ns, total: 28.7 s
Wall time: 30.7 s


In [155]:
#Получаем данные для пользователей для которых надо удет предсказать пол и возраст
public_test = id_to_submit.merge(info_user50,on=['user_id'],how='left')

In [157]:
usr_targets = targets.to_pandas()
info_user50 = info_user50.reset_index()



In [158]:
#Полчаем данные для обучения моделей с имеющимися таргетами
df = usr_targets.merge(info_user50, how = 'inner', on = ['user_id'])
df_train=df[df['is_male'] != 'NA']
df_train=df_train.dropna()
df_train['is_male'] = df_train['is_male'].astype('int')

In [159]:
categorical = ['reg_main','cpe_manufacturer_name','cpe_type_cd','cpe_model_os_type','cpe_model_name']
numerical = []
for i in df_train.columns:
    if i not in [*categorical,'age','is_male','user_id']:
        numerical.append(i)

In [160]:
df_train_puplic=df_train.drop(['user_id'],axis=1)

In [161]:
df_train_puplic_features = df_train_puplic.drop(['is_male','age'],axis=1)
df_train_puplic_target_is_male = df_train_puplic['is_male']
def age_bucket(x):
    return bisect.bisect_left([18,25,35,45,55,65], x)

df_train_puplic_target_age_buck = df_train_puplic['age'].map(age_bucket)

In [162]:
df_test_puplic_features = public_test.drop(['user_id'],axis=1)

In [163]:
#нормализуем численные параметры
scaler = StandardScaler()
scaler.fit(df_train_puplic_features[numerical])
df_train_puplic_features[numerical]=scaler.transform(df_train_puplic_features[numerical])
df_test_puplic_features[numerical]=scaler.transform(df_test_puplic_features[numerical])

/home/admin/anaconda3/lib/python3.9/site-packages/sklearn/utils/validation.py:624: UserWarning: pandas.DataFrame with sparse columns found.It will be converted to a dense numpy array.
  warnings.warn(
/home/admin/anaconda3/lib/python3.9/site-packages/sklearn/utils/validation.py:624: UserWarning: pandas.DataFrame with sparse columns found.It will be converted to a dense numpy array.
  warnings.warn(
/home/admin/anaconda3/lib/python3.9/site-packages/sklearn/utils/validation.py:624: UserWarning: pandas.DataFrame with sparse columns found.It will be converted to a dense numpy array.
  warnings.warn(


In [166]:
#проводим перекодировку категориальных переменных
df_train_puplic_features_oe=df_train_puplic_features.copy()
df_test_puplic_featuress_oe=df_test_puplic_features.copy()



In [167]:
oe=OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=-1)
oe.fit(df_train_puplic_features_oe[categorical])
df_train_puplic_features_oe[categorical]=oe.transform(df_train_puplic_features_oe[categorical])
df_test_puplic_featuress_oe[categorical]=oe.transform(df_test_puplic_featuress_oe[categorical])

In [168]:
df_train_puplic_features_oe[categorical]=df_train_puplic_features_oe[categorical].astype(str)
df_test_puplic_featuress_oe[categorical]=df_test_puplic_featuress_oe[categorical].astype(str)

In [170]:
#Предсказываем пол 
%%time
model = CatBoostClassifier()
model.fit(df_train_puplic_features_oe,df_train_puplic_target_is_male, verbose = False,cat_features=categorical)
predict_proba_is_male=model.predict_proba(df_test_puplic_featuress_oe)[:,1]


CPU times: user 32min 57s, sys: 1min 48s, total: 34min 46s
Wall time: 3min 10s


In [172]:
#Предсказываем возраст
%%time
model = CatBoostClassifier()
model.fit(df_train_puplic_features_oe,df_train_puplic_target_age_buck, verbose = False,cat_features=categorical)
predict_age=model.predict(df_test_puplic_featuress_oe)


CPU times: user 4h 7min 59s, sys: 7min 59s, total: 4h 15min 58s
Wall time: 22min 58s


In [174]:
#Сохраняем результат
result = pd.DataFrame(zip(public_test['user_id'],predict_age,predict_proba_is_male),columns = ['user_id','age','is_male'])

In [175]:
result['age']=result['age'].astype(int)

In [176]:
result

,user_id,age,is_male
0,221301,2,0.957293
1,31271,3,0.574391
2,211594,2,0.539828
3,253119,2,0.515981
4,192578,2,0.689335
...,...,...,...
144719,415284,2,0.519580
144720,415285,3,0.548605
144721,415286,3,0.482689
144722,415306,2,0.630096


In [177]:
id_to_submit = pq.read_table(f'{LOCAL_DATA_PATH}/{SUBMISSION_FILE}').to_pandas()

In [178]:
id_to_submit=id_to_submit.merge(result, on=['user_id'],how='left')

In [179]:
id_to_submit=id_to_submit.set_index('user_id')

In [180]:
id_to_submit.to_csv('submission_kate_ng500.csv')

## Подбор Гиперараметров

In [66]:
train, test = train_test_split(df_train,test_size = 0.25,random_state=12345)


train_traget_age = train['age']
train_traget_age_buck = train_traget_age.map(age_bucket)
train_traget_is_male = train['is_male']

test_traget_age = test['age']
test_traget_age_buck = test_traget_age.map(age_bucket)
test_traget_is_male = test['is_male']

train_features= train.drop(['age','is_male','user_id'],axis=1)
test_features= test.drop(['age','is_male','user_id'],axis=1)





In [69]:
scaler = StandardScaler()
scaler.fit(train_features[numerical])
train_features[numerical]=scaler.transform(train_features[numerical])
test_features[numerical]=scaler.transform(test_features[numerical])

/home/admin/anaconda3/lib/python3.9/site-packages/sklearn/utils/validation.py:624: UserWarning: pandas.DataFrame with sparse columns found.It will be converted to a dense numpy array.
  warnings.warn(
/home/admin/anaconda3/lib/python3.9/site-packages/sklearn/utils/validation.py:624: UserWarning: pandas.DataFrame with sparse columns found.It will be converted to a dense numpy array.
  warnings.warn(
/home/admin/anaconda3/lib/python3.9/site-packages/sklearn/utils/validation.py:624: UserWarning: pandas.DataFrame with sparse columns found.It will be converted to a dense numpy array.
  warnings.warn(


In [73]:
train_features_oe=train_features.copy()
test_features_oe=test_features.copy()

oe=OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=-1)
oe.fit(train_features_oe[categorical])

train_features_oe[categorical]=oe.transform(train_features_oe[categorical])
test_features_oe[categorical]=oe.transform(test_features_oe[categorical])

train_features_oe[categorical]=train_features_oe[categorical].astype('str')
test_features_oe[categorical]=test_features_oe[categorical].astype('str')

In [85]:
model = CatBoostClassifier()

In [96]:
params={'iterations':[700,1000,1400],
'learning_rate':[0.01,0.03,0.1],
 'max_depth':[6,12,14,16],
 'n_estimators':[100,300,500]
 }


In [98]:
%%time
max_gini=-1111
for iterations in params['iterations']:
    for l_rate in params['learning_rate']:
        for max_depth in params['max_depth']:
            model=CatBoostClassifier(iterations=iterations,\
                                         learning_rate = l_rate,\
                                        max_depth = max_depth)
            model.fit(train_features_oe,train_traget_is_male, verbose = False,cat_features=categorical)
            predict_proba=model.predict_proba(test_features_oe)[:,1]
            gini_CB_gender=2 * m.roc_auc_score(test_traget_is_male, predict_proba)
            if gini_CB_gender>max_gini:
                max_gini=gini_CB_gender
                best_it = iterations
                best_l_r=l_rate
                best_m_depth=max_depth
                
   

KeyboardInterrupt: 

In [104]:
%%time
model = CatBoostClassifier(iterations=best_it,\
                                         learning_rate = best_l_r,\
                                        max_depth = best_m_depth)
model.fit(train_features_oe,train_traget_is_male, verbose = False,cat_features=categorical)
predict_proba=model.predict_proba(test_features_oe)[:,1]
gini_CB_gender=2 * m.roc_auc_score(test_traget_is_male, predict_proba)
CB_params_gender=""
print(f'GINI по полу {gini_CB_gender - 1:2.3f}')

KeyboardInterrupt: 

In [ ]:
%%time
model = CatBoostClassifier(iterations=best_iterations,\
                                         learning_rate = best_l_r,\
                                        max_depth = best_m_depth)
model.fit(df_train_puplic_features_oe,df_train_puplic_target_is_male, verbose = False,cat_features=categorical)
predict_proba_is_male=model.predict_proba(df_test_puplic_featuress_oe)[:,1]


In [112]:
params2={'iterations':[700,1000,1400],
'learning_rate':[0.01,0.03],
 'max_depth':[6,12,16]
 }

In [117]:
%%time
f1max=-1111
#for iterations in params2['iterations']:
for l_rate in params2['learning_rate']:
    for max_depth in params2['max_depth']:
        model=CatBoostClassifier(#iterations=iterations,\
                                         learning_rate = l_rate,\
                                        max_depth = max_depth)
        model.fit(train_features_oe,train_traget_age_buck, verbose = False,cat_features=categorical)
        predict=model.predict(test_features_oe)
        f1=f1_score(test_traget_age_buck, predict,average='weighted')
        if f1>f1max:
            f1max=f1
            #best_it2 = iterations
            best_l_r2=l_rate
            best_m_depth2=max_depth

KeyboardInterrupt: 

In [ ]:
%%time
model = CatBoostClassifier(iterations=best_it2,\
                                         learning_rate = best_l_r2,\
                                        max_depth = best_m_depth2)
model.fit(train_features_oe,train_traget_age_buck,verbose = False,cat_features=categorical)

print(m.classification_report(test_traget_age_buck, model.predict(test_features_oe), \
                            target_names = ['<18', '18-25','25-34', '35-44', '45-54', '55-65', '65+']))

In [ ]:
%%time
model = CatBoostClassifier(iterations=best_it2,\
                                         learning_rate = best_l_r2,\
                                        max_depth = best_m_depth2)
model.fit(df_train_puplic_features_oe,df_train_puplic_target_age_buck, verbose = False,cat_features=categorical)
predict_age=model.predict(df_test_puplic_featuress_oe)


In [ ]:
best_it2

In [ ]:
best_l_r2

In [ ]:
best_m_depth2

In [82]:
%%time
model = XGBClassifier()
model.fit(train_features_oe,train_traget_is_male, verbose = False)
predict_proba=model.predict_proba(test_features_oe)[:,1]
gini_CB_gender=2 * m.roc_auc_score(test_traget_is_male, predict_proba)
CB_params_gender=""
print(f'GINI по полу {gini_CB_gender - 1:2.3f}')

GINI по полу 0.713
CPU times: user 19min 13s, sys: 7.02 s, total: 19min 20s
Wall time: 1min 22s


In [83]:
%%time
model = XGBClassifier()
model.fit(train_features_oe,train_traget_age_buck,verbose = False)

print(m.classification_report(test_traget_age_buck, model.predict(test_features_oe), \
                            target_names = ['<18', '18-25','25-34', '35-44', '45-54', '55-65', '65+']))

              precision    recall  f1-score   support

         <18       0.00      0.00      0.00       210
       18-25       0.55      0.36      0.43      7966
       25-34       0.49      0.65      0.56     21340
       35-44       0.41      0.52      0.46     19093
       45-54       0.38      0.21      0.27     10183
       55-65       0.43      0.22      0.29      5934
         65+       0.35      0.04      0.07      1356

    accuracy                           0.45     66082
   macro avg       0.37      0.28      0.30     66082
weighted avg       0.45      0.45      0.43     66082

CPU times: user 2h 9min 2s, sys: 31.7 s, total: 2h 9min 33s
Wall time: 9min 3s
